# ColabLink Setup

This notebook sets up your Google Colab runtime to accept connections from your local machine.

## What This Does

1. Installs ColabLink on this Colab instance
2. Sets up SSH server for secure connection
3. Creates a tunnel so your local machine can connect
4. Provides connection instructions

## After Setup

You'll be able to:
- Run code from your local terminal on this Colab GPU
- Keep all your files local
- See real-time output in your local terminal
- Use VS Code, Cursor, or any IDE locally

---

## Instructions

1. Run the cell below
2. Copy the connection command from the output
3. Paste it in your local terminal
4. Start executing code on Colab GPU from your local machine!

Keep this notebook running to maintain the connection.


In [ ]:
# Optional: Fix broken packages (if you encounter installation errors)
# Uncomment and run this cell if the setup fails with package errors
# !apt-get update
# !dpkg --configure -a
# !apt-get install -y -f


In [ ]:
# Install ColabLink (choose one option)
use_github = False  # Set True to install latest from GitHub main branch

if use_github:
    !pip install -q git+https://github.com/PoshSylvester/colablink.git
else:
    !pip install -q colablink

print("ColabLink installed successfully!")


In [ ]:
from getpass import getpass
from colablink import ColabRuntime

# Configuration
PASSWORD = getpass('Enter a strong password for the SSH user (leave blank to autogenerate): ')
NGROK_TOKEN = getpass('Enter your ngrok authtoken: ').strip()
USERNAME = 'colablink'  # Lowercase letters/digits/_/- only
REMOTE_ROOT = '/content'  # Base directory for all connections

if not NGROK_TOKEN:
    raise ValueError('ngrok authtoken is required. Get one at https://dashboard.ngrok.com/get-started/your-authtoken')

# Setup runtime
runtime = ColabRuntime(
    password=PASSWORD or None,
    ngrok_token=NGROK_TOKEN,
    username=USERNAME,
    remote_root=REMOTE_ROOT,
)

# Run setup
print("Setting up ColabLink runtime...")
print("This will:")
print("- Create SSH user with /content write permissions")
print("- Start SSH server")
print("- Create secure tunnel")
print()

connection_info = runtime.setup()

print()
print("🎉 Setup complete!")
print()

# Create connection directories as needed
print("Creating connection directories...")
connection_dir1 = runtime.create_connection_dir("connection_main")
connection_dir2 = runtime.create_connection_dir()  # Auto-generated name

# Update connection info with the created directories
connection_info["remote_output_dir"] = connection_dir1  # Default to first one
print(f"Available connection directories: {runtime.connection_dirs}")
print()

print("Connection info for local machine:")
import json
print(json.dumps(connection_info, indent=2))
print()

# Keep the runtime alive
# This cell will keep running - don't interrupt it!
runtime.keep_alive()


## Get an ngrok Authtoken

ngrok now requires authentication for all tunnels. Before running the setup cell:

1. Visit https://dashboard.ngrok.com/signup and create a free account (verify your email).
2. Copy your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken.
3. Paste it when prompted in the setup cell above.

## Next Steps

After setup completes:

1. Install colablink on your local machine (pick one):
   ```bash
   pip install colablink              # PyPI release
   pip install git+https://github.com/PoshSylvester/colablink.git  # Latest main
   ```

2. Copy the connection JSON from above and run it locally. Example:
   ```bash
   colablink init '{"host": "...", "port": "...", "username": "colablink", "password": "...", "remote_root": "/content", "remote_output_dir": "/content/connection_main"}' \
     --local-dir connection_main \
     --profile main
   ```

3. Start using Colab GPU from your local terminal:
   ```bash
   colablink exec python train.py
   colablink exec nvidia-smi
   ```

4. Or start an interactive shell:
   ```bash
   colablink shell
   ```

Keep this notebook running to maintain the tunnel.
